In [1]:
import os
import chromadb
from langchain.vectorstores import FAISS
# from langchain.text_splitter import SpacyTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain_openai import OpenAIEmbeddings

# from langchain_community.embeddings import OpenAIEmbeddings

from langchain_community.vectorstores import chroma
from langchain_community.vectorstores import Chroma
from langchain.document_transformers import (
    LongContextReorder,
)
from langchain_text_splitters import CharacterTextSplitter

from langchain.retrievers import MergerRetriever, ContextualCompressionRetriever


In [2]:
embeddings = OpenAIEmbeddings()
print("Embeddings loaded")

Embeddings loaded


# Data Presets using Chroma DB

In [8]:
# Loading Resume
resume = PyPDFLoader("Testing/resume/VishnuPrakash_Resume.pdf")
resume_load = resume.load()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap=0)
resume_splitted = text_splitter.split_documents(resume_load)

print(resume_splitted)

In [7]:
# loading job description
path = "Testing/resume/job_desc.txt"
with open(path, "r") as file:
    job_desc = file.read()

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap=0)
jd_splitted = text_splitter.split_text(job_desc)

print(jd_splitted[0]) 

As a Junior Machine Learning Engineer at our cutting-edge company, your role is crucial in developing the future of AI chatbots and Large Language Models (LLMs) like Llama2, with a special emphasis on integrating these innovations with WIX platforms. Your work will involve a deep dive into AWS services, Data ETL, GitHub for code collaboration, and the hands-on building of models, all while leveraging your strong quantitative and programming background. This role demands not only technical expertise but also a creative approach to problem-solving and code development. You will work in close partnership with the CTO, ensuring that your contributions leave a lasting impact on our AI-driven solutions. Additionally, your collaboration with front-end developers is key to weaving AI functionalities into WIX-based applications seamlessly, enhancing user experiences and pushing the boundaries of what our digital platforms can achieve.


## Create Vector Storage

In [7]:
# vectorDB for resume
chroma_resume = Chroma.from_documents(
        resume_splitted,embeddings,
        collection_metadata={"hnsw:space": "cosine"},persist_directory="vector_storage/resume_store" # l2 is the default
    )
# vectorDB for JD
chroma_jd = Chroma.from_texts(
    jd_splitted,embeddings,collection_metadata = {"hnsw:space": "cosine"},
    persist_directory="vector_storage/jd_store" 
)

## Load Vector stores


In [11]:
load_chroma_resume = Chroma(persist_directory="vector_storage/resume_store",embedding_function=embeddings)
load_chroma_jd = Chroma(persist_directory="vector_storage/jd_store",embedding_function=embeddings)

## Initialize Merge Retriever and Perform semantic search

In [21]:
retriever_resume = load_chroma_resume.as_retriever(search_type = "similarity", search_kwargs = {"k":1})

retriever_jd = load_chroma_jd.as_retriever(search_type = "similarity", search_kwargs = {"k":1})


In [22]:
merged = MergerRetriever(retrievers=[retriever_resume, retriever_jd])


In [23]:
for chunks in merged.get_relevant_documents("Machine Learning"):
    print(chunks.page_content)

VISHNU PRAKASH
Data Scientist
vishnucheppanam@gmail.com — +91 80 780 43398
linkedin.com/in/vishnuprksh — github.com/vishnuprksh — vishnuprakash.online
PROFILE
Data Science aspirant with hands-on experience in machine learning and deep learning, seeking a
challenging role as a Data Scientist/Machine Learning Engineer. Eager to leverage skills in Artificial
Intelligence, Neural Networks, and Algorithms to contribute to the companies and personal growth.
PROJECTS
GemInsights Git Link , Automating Exploratory Data Analysis (EDA) and Insight generation with
the help of latest Gemini engine by Google
•EDA generation with the help of AutoViz and Image analysis with Gemini-Pro-Vision.
•Hallucination check with Trulense-Eval.
•UI with Streamlit.
TradePilot Git Link , Empowering stock market price prediction with sentiment analysis from News,
Twitter and Reddit
•Stock price history collected from Yahoo Finance and Text data scraped from Economic Times,
Reddit, and Twitter.
•Sentiment analysis wi